<a href="https://colab.research.google.com/github/quocbaodinhnho/BoBiu/blob/master/baitaptuan3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Bao Tap lop 1
import math

PLAYER = "X"   # AI
OPPONENT = "O" # Người chơi

# Kiểm tra chiến thắng
def check_winner(board):
    win_states = [
        [0,1,2], [3,4,5], [6,7,8],    # hàng
        [0,3,6], [1,4,7], [2,5,8],    # cột
        [0,4,8], [2,4,6]              # đường chéo
    ]
    for a, b, c in win_states:
        if board[a] == board[b] == board[c] != " ":
            return board[a]
    return None

# Kiểm tra hòa
def is_draw(board):
    return " " not in board and check_winner(board) is None

# Thuật toán Minimax
def minimax(board, depth, is_maximizing):
    winner = check_winner(board)
    if winner == PLAYER:
        return 10 - depth
    if winner == OPPONENT:
        return depth - 10
    if is_draw(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == " ":
                board[i] = PLAYER
                score = minimax(board, depth + 1, False)
                board[i] = " "
                best_score = max(best_score, score)
        return best_score
    else:
        best_score = math.inf
        for i in range(9):
            if board[i] == " ":
                board[i] = OPPONENT
                score = minimax(board, depth + 1, True)
                board[i] = " "
                best_score = min(best_score, score)
        return best_score

# Tìm nước đi tốt nhất cho AI
def find_best_move(board):
    best_score = -math.inf
    best_move = -1

    for i in range(9):
        if board[i] == " ":
            board[i] = PLAYER
            score = minimax(board, 0, False)
            board[i] = " "
            if score > best_score:
                best_score = score
                best_move = i

    return best_move


In [2]:
#Ba tap lop 2
import math

PLAYER = "X"     # AI
OPPONENT = "O"   # Người chơi

def alpha_beta(board, depth, alpha, beta, is_maximizing):
    winner = check_winner(board)
    if winner == PLAYER:
        return 10 - depth
    if winner == OPPONENT:
        return depth - 10
    if is_draw(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == " ":
                board[i] = PLAYER
                score = alpha_beta(board, depth+1, alpha, beta, False)
                board[i] = " "
                best_score = max(best_score, score)
                alpha = max(alpha, best_score)
                if beta <= alpha:   # CẮT TỈA
                    break
        return best_score
    else:
        best_score = math.inf
        for i in range(9):
            if board[i] == " ":
                board[i] = OPPONENT
                score = alpha_beta(board, depth+1, alpha, beta, True)
                board[i] = " "
                best_score = min(best_score, score)
                beta = min(beta, best_score)
                if beta <= alpha:   # CẮT TỈA
                    break
        return best_score

# Tìm nước đi tốt nhất
def best_move_alpha_beta(board):
    best_score = -math.inf
    best_move = -1

    for i in range(9):
        if board[i] == " ":
            board[i] = PLAYER
            score = alpha_beta(board, 0, -math.inf, math.inf, False)
            board[i] = " "
            if score > best_score:
                best_score = score
                best_move = i

    return best_move


In [ ]:
import copy
import math
import numpy

X = "X"
O = "O"
EMPTY = None

user = None
ai = None

def initial_state():
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

# ----- FIXED -----
def player(board):
    """
    Returns player who has the next turn on a board.
    X always moves first.
    """
    count_X = sum(row.count(X) for row in board)
    count_O = sum(row.count(O) for row in board)
    return X if count_X == count_O else O


def actions(board):
    res = set()
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res


def result(board, action):
    new = copy.deepcopy(board)
    i, j = action
    new[i][j] = player(board)
    return new


def get_horizontal_winner(board):
    for row in board:
        if row[0] is not None and row.count(row[0]) == 3:
            return row[0]
    return None


def get_vertical_winner(board):
    for col in range(3):
        col_vals = [board[r][col] for r in range(3)]
        if col_vals[0] is not None and col_vals.count(col_vals[0]) == 3:
            return col_vals[0]
    return None


def get_diagonal_winner(board):
    if board[0][0] is not None and board[0][0] == board[1][1] == board[2][2]:
        return board[0][0]
    if board[0][2] is not None and board[0][2] == board[1][1] == board[2][0]:
        return board[0][2]
    return None


def winner(board):
    return get_horizontal_winner(board) or \
           get_vertical_winner(board) or \
           get_diagonal_winner(board)


def terminal(board):
    if winner(board) is not None:
        return True
    return all(all(cell is not EMPTY for cell in row) for row in board)


def utility(board):
    w = winner(board)
    if w == X:
        return 1
    elif w == O:
        return -1
    return 0


def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v


def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v


# -------- FIXED minimax --------
def minimax(board):
    turn = player(board)

    best_move = None

    if turn == X:
        best_score = -math.inf
        for action in actions(board):
            score = minValue(result(board, action))
            if score > best_score:
                best_score = score
                best_move = action
    else:
        best_score = math.inf
        for action in actions(board):
            score = maxValue(result(board, action))
            if score < best_score:
                best_score = score
                best_move = action

    return best_move
# --------------------------------


if __name__ == "__main__":
    board = initial_state()

    print("Choose your side (X or O):")
    user = input().strip().upper()
    while user not in ["X", "O"]:
        user = input("Choose X or O: ").strip().upper()

    ai = O if user == X else X

    print("You are:", user, "| AI is:", ai)
    print(numpy.array(board))

    while True:
        if terminal(board):
            w = winner(board)
            if w is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {w} wins.")
            break

        if player(board) == user:
            # player turn
            print("Your turn (0-2):")
            try:
                r = int(input("Row: "))
                c = int(input("Col: "))
            except:
                print("Invalid input.")
                continue

            if (r not in range(3)) or (c not in range(3)):
                print("Invalid move.")
                continue
            if board[r][c] != EMPTY:
                print("Cell already taken.")
                continue

            board = result(board, (r, c))
            print(numpy.array(board))

        else:
            print("AI thinking...")
            move = minimax(board)
            board = result(board, move)
            print("AI moves:", move)
            print(numpy.array(board))


Choose your side (X or O):


In [6]:
#Bai3
import math

PLAYER = "X"
OPPONENT = "O"

def check_winner(board, N, K):
    """Kiểm tra thắng theo hàng, cột, chéo (K liên tiếp)."""
    lines = []


    for r in range(N):
        lines.append(board[r])


    for c in range(N):
        col = [board[r][c] for r in range(N)]
        lines.append(col)


    for start in range(N - K + 1):
        for c in range(N - K + 1):
            diag = [board[start + i][c + i] for i in range(K)]
            lines.append(diag)


    for start in range(N - K + 1):
        for c in range(K - 1, N):
            diag = [board[start + i][c - i] for i in range(K)]
            lines.append(diag)


    for line in lines:
        if all(cell == PLAYER for cell in line):
            return PLAYER
        if all(cell == OPPONENT for cell in line):
            return OPPONENT

    return None



def evaluate(board, N, K):
    win = check_winner(board, N, K)
    if win == PLAYER:
        return 100000
    if win == OPPONENT:
        return -100000
    return 0


def minimax(board, depth, is_max, N, K, max_depth):
    score = evaluate(board, N, K)

    if abs(score) == 100000:
        return score - depth if score > 0 else score + depth

    if depth == max_depth:
        return score  # giới hạn độ sâu

    if is_max:
        best = -math.inf
        for r in range(N):
            for c in range(N):
                if board[r][c] == " ":
                    board[r][c] = PLAYER
                    best = max(best, minimax(board, depth + 1, False, N, K, max_depth))
                    board[r][c] = " "
        return best

    else:  # Min
        best = math.inf
        for r in range(N):
            for c in range(N):
                if board[r][c] == " ":
                    board[r][c] = OPPONENT
                    best = min(best, minimax(board, depth + 1, True, N, K, max_depth))
                    board[r][c] = " "
        return best


def best_move_minimax(board, N, K, max_depth=2):
    best_score = -math.inf
    move = (-1, -1)

    for r in range(N):
        for c in range(N):
            if board[r][c] == " ":
                board[r][c] = PLAYER
                score = minimax(board, 0, False, N, K, max_depth)
                board[r][c] = " "
                if score > best_score:
                    best_score = score
                    move = (r, c)
    return move


In [1]:
#Bai4
def alphabeta(board, depth, alpha, beta, is_max, N, K, max_depth):
    score = evaluate(board, N, K)

    if abs(score) == 100000:
        return score

    if depth == max_depth:
        return score

    if is_max:
        best = -math.inf
        for r in range(N):
            for c in range(N):
                if board[r][c] == " ":
                    board[r][c] = PLAYER
                    val = alphabeta(board, depth+1, alpha, beta, False, N, K, max_depth)
                    board[r][c] = " "
                    best = max(best, val)
                    alpha = max(alpha, best)
                    if beta <= alpha:
                        break
        return best

    else:
        best = math.inf
        for r in range(N):
            for c in range(N):
                if board[r][c] == " ":
                    board[r][c] = OPPONENT
                    val = alphabeta(board, depth+1, alpha, beta, True, N, K, max_depth)
                    board[r][c] = " "
                    best = min(best, val)
                    beta = min(beta, best)
                    if beta <= alpha:
                        break
        return best


def best_move_alphabeta(board, N, K, max_depth=3):
    best_score = -math.inf
    best_move = (-1, -1)

    for r in range(N):
        for c in range(N):
            if board[r][c] == " ":
                board[r][c] = PLAYER
                score = alphabeta(board, 0, -math.inf, math.inf, False, N, K, max_depth)
                board[r][c] = " "
                if score > best_score:
                    best_score = score
                    best_move = (r, c)

    return best_move


In [2]:
#Bai 5
def alphabeta_chess(state, depth, alpha, beta, is_max):
    if depth == 0 or state.is_terminal():
        return evaluate_chess(state)

    moves = generate_moves(state)
    order_moves(moves)    # sắp xếp nước đi tốt trước

    if is_max:
        value = -inf
        for move in moves:
            new_state = apply_move(state, move)
            value = max(value, alphabeta_chess(new_state, depth-1, alpha, beta, False))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value
    else:
        value = +inf
        for move in moves:
            new_state = apply_move(state, move)
            value = min(value, alphabeta_chess(new_state, depth-1, alpha, beta, True))
            beta = min(beta, value)
            if alpha >= beta:
                break
        return value
